# 모의고사 1회

## 작업형1(은행 전화 마케팅에 대해 고객의 반응여부)

In [4]:
import pandas as pd
data= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv')

data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y
0,13829,29,technician,single,tertiary,no,18254,no,no,cellular,11,may,2,-1,0,unknown,no
1,22677,26,services,single,secondary,no,512,yes,yes,unknown,5,jun,3,-1,0,unknown,no
2,10541,30,management,single,secondary,no,135,no,no,cellular,14,aug,2,-1,0,unknown,no
3,13689,41,technician,married,unknown,no,30,yes,no,cellular,10,jul,1,-1,0,unknown,no
4,11304,27,admin.,single,secondary,no,321,no,yes,unknown,2,sep,1,-1,0,unknown,no


In [6]:
# 마케팅 응답 고객들의 나이를 10살 단위로 변환 했을 때, 가장 많은 인원을 가진 나이대는? (0~9 : 0 , 10~19 : 10)

ans = (data['age']//10*10).value_counts().index[0]
print(ans)

30


In [12]:
# 마케팅 응답 고객들의 나이를 10살 단위로 변환 했을 때, 가장 많은 나이대 구간의 인원은 몇명인가?

ans=(data['age']//10*10).value_counts().values[0]
print(ans)

5056


In [16]:
# 나이가 25살 이상 29살 미만인 응답 고객들중 housing컬럼의 값이 yes인 고객의 수는?

ans = len(data[(25<=data['age'])&(data['age']<29)&(data['housing']=='yes')])
print(ans)

504


In [35]:
# numeric한 값을 가지지 않은 컬럼들중 unique한 값을 가장 많이 가지는 컬럼은?
lst= [] 
for col in data.select_dtypes(exclude='int'):
    target = data[col]
    lst.append([col,target.nunique()])

tmp = pd.DataFrame(lst).sort_values(1,ascending=False).values[0][0]
print(tmp)

job


In [45]:
# balance 컬럼값들의 평균값 이상을 가지는 데이터를 ID값을 기준으로 내림차순 정렬했을때 상위 100개 데이터의 balance값의 평균은?

ans = data[data['balance']>=data['balance'].mean()].sort_values('ID',ascending=False).iloc[:100]['balance'].mean()
print(ans)

3473.73


In [54]:
# 가장 많은 광고를 집행했던 날짜는 언제인가? (데이터 그대로 일(숫자),달(영문)으로 표기)

ans = data[['day','month']].value_counts().index[0]
print(ans)

(15, 'may')


In [60]:
# 데이터의 job이 unknown 상태인 고객들의 age 컬럼 값의 정규성을 검정하고자 한다. 샤피로 검정의 p-value값을 구하여라

from scipy import stats
ans = stats.shapiro(data[data['job']=='unknown']['age'])[1]
print(ans)

0.1961131989955902


In [64]:
# age와 balance의 상관계수를 구하여라

ans = data.corr().iloc[1,2]
print(ans)

0.10198734763981462


In [68]:
# y 변수와 education 변수는 독립인지 카이제곱검정을 통해 확인하려한다. p-value값을 출력하라

v = pd.crosstab(data['y'],data['education'])
from scipy.stats import chi2_contingency
chi2,p,dof,expected = chi2_contingency(v)
print(p)

7.901201277473551e-29


In [78]:
# 각 job에 따라 divorced/married 인원의 비율을 확인 했을 때 그 값이 가장 높은 값은?
tmp = data.groupby(['job','marital']).size().unstack()
tmp['ratio']=tmp['divorced']/tmp['married']
tmp['ratio'].sort_values(ascending=False).values[0]

0.2831050228310502

## 작업형 2

In [114]:
import pandas as pd
train= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv')
test= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/test.csv')
submission= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/submission.csv')

display(train.head())
display(test.head())
display(submission.head())

train.drop('ID',axis=1,inplace=True)
x_test_id = test.pop('ID')
y = train.pop('y')

# print(train.info())
# print(train.describe())

for col in train.columns:
    print(f'{col}의 unique : \n{train[col].unique()}')
    print('-'*80)
str_col = train.select_dtypes('object').columns
num_col = train.select_dtypes(exclude='object').columns

print('여기는 str_column 값 확인용')
one_col=[]
le_col = []

for col in str_col:
    print(f'{col}의 unique : \n {train[col].nunique()}')
    if train[col].nunique()<4:
        one_col.append(col)
    else:
        le_col.append(col)

from sklearn.preprocessing import StandardScaler,LabelEncoder,RobustScaler

le=LabelEncoder()

for col in le_col:
    train[col]=le.fit_transform(train[col])
    test[col]=le.fit_transform(test[col])

train=pd.get_dummies(train,columns=one_col)
test=pd.get_dummies(test,columns=one_col)


scaler = StandardScaler()

train=scaler.fit_transform(train)
test=scaler.fit_transform(test)

from sklearn.model_selection import train_test_split
x_tr,x_te,y_tr,y_te = train_test_split(train,y,test_size=0.2)

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

model_rf = RandomForestClassifier()
model_rf.fit(x_tr,y_tr)
pred_rf = model_rf.predict(x_te)
score_rf = model_rf.score(x_te,y_te)

# model_xgb = XGBClassifier()
# model_xgb.fit(x_tr,y_tr)
# pred_xgb = model_xgb.predict(x_te)
# score_xgb = model_xgb.score(x_te,y_te)

model_dt = DecisionTreeClassifier()
model_dt.fit(x_tr,y_tr)
pred_dt = model_dt.predict(x_te)
score_dt = model_dt.score(x_te,y_te)

model_svc = SVC()
model_svc.fit(x_tr,y_tr)
pred_svc = model_svc.predict(x_te)
score_svc = model_svc.score(x_te,y_te)

print(f'rf score : {score_rf}')
# print(f'xgb score : {score_xgb}')
print(f'dt score : {score_dt}')
print(f'svc score : {score_svc}')

from sklearn.metrics import classification_report,roc_auc_score

report1 = classification_report(y_te,pred_rf)
report2 = classification_report(y_te,pred_dt)
report3 = classification_report(y_te,pred_svc)

print(report1)
print(report2)
print(report3)
final_pred = model_rf.predict_proba(test)
print(final_pred)
print(final_pred[:,1])
submission['predict']=final_pred[:,1]
display(submission.head())
# output = pd.DataFrame({'ID':x_test_id,'predict':final_pred })
# output.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y
0,13829,29,technician,single,tertiary,no,18254,no,no,cellular,11,may,2,-1,0,unknown,no
1,22677,26,services,single,secondary,no,512,yes,yes,unknown,5,jun,3,-1,0,unknown,no
2,10541,30,management,single,secondary,no,135,no,no,cellular,14,aug,2,-1,0,unknown,no
3,13689,41,technician,married,unknown,no,30,yes,no,cellular,10,jul,1,-1,0,unknown,no
4,11304,27,admin.,single,secondary,no,321,no,yes,unknown,2,sep,1,-1,0,unknown,no


,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome
0,53608,32,management,single,tertiary,no,12569,no,no,cellular,1,jul,2,295,2,success
1,51055,25,services,single,secondary,no,801,no,no,cellular,5,jun,2,-1,0,unknown
2,52573,46,blue-collar,married,secondary,no,1728,yes,no,unknown,26,may,2,-1,0,unknown
3,50458,39,management,divorced,secondary,no,51,no,no,unknown,17,jun,2,-1,0,unknown
4,52272,31,services,single,tertiary,no,1626,no,no,unknown,31,jul,1,-1,0,unknown


,ID,predict
0,53608,0.0
1,51055,0.0
2,52573,0.0
3,50458,0.0
4,52272,0.0


age의 unique : 
[29 26 30 41 27 22 53 38 59 37 47 48 90 36 31 52 39 58 35 20 28 93 25 43
 51 55 44 56 49 32 33 40 74 60 62 45 57 82 46 65 34 67 50 63 42 69 64 77
 54 24 23 81 68 66 73 88 19 70 61 86 84 72 79 21 75 76 71 18 78 83 85 80
 87 95 89 92]
--------------------------------------------------------------------------------
job의 unique : 
['technician' 'services' 'management' 'admin.' 'student' 'blue-collar'
 'housemaid' 'retired' 'unemployed' 'self-employed' 'unknown'
 'entrepreneur']
--------------------------------------------------------------------------------
marital의 unique : 
['single' 'married' 'divorced']
--------------------------------------------------------------------------------
education의 unique : 
['tertiary' 'secondary' 'unknown' 'primary']
--------------------------------------------------------------------------------
default의 unique : 
['no' 'yes']
--------------------------------------------------------------------------------
balance의 unique : 
[18254   512  

,ID,predict
0,53608,0.82
1,51055,0.78
2,52573,0.06
3,50458,0.15
4,52272,0.27


# 모의고사 2회

## 작업형1(뇌졸증 발생여부 예측)

In [232]:
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/stroke_/train.csv')
data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1192,Female,31,0,0,No,Govt_job,Rural,70.66,27.2,never smoked,0
1,77,Female,13,0,0,No,children,Rural,85.81,18.6,Unknown,0
2,59200,Male,18,0,0,No,Private,Urban,60.56,33.0,never smoked,0
3,24905,Female,65,0,0,Yes,Private,Urban,205.77,46.0,formerly smoked,1
4,24257,Male,4,0,0,No,children,Rural,90.42,16.2,Unknown,0


In [210]:
#성별이 Male인 환자들의 age의 평균값은 ?

data['age'] = data['age'].str.replace('*','').astype('int')
result = data[data.gender =='Male'].age.mean()
print(result)

44.68623481781376


C:\Users\SuperUser\AppData\Local\Temp\ipykernel_11596\2246737320.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['age'] = data['age'].str.replace('*','').astype('int')


In [211]:
# bmi컬럼의 결측치를 bmi컬럼의 결측치를 제외한 나머지 값들의 중앙값으로 채웠을 경우 bmi 컬럼의 평균을 소숫점 이하 3자리 까지 구하여라

fi = data['bmi'].fillna(data['bmi'].median())
result = round(fi.mean(),3)
print(result)

29.166


In [212]:
# bmi컬럼의 각 결측치들을 직전의 행의 bmi값으로 채웠을 경우 bmi 컬럼의 평균을 소숫점 이하 3자리 까지 구하여라
ans1 = round(data['bmi'].fillna(method='ffill').mean(),3)
print(ans1)

29.188


In [215]:
# bmi컬럼의 각 결측치들을 결측치를 가진 환자 나이대(10단위)의 평균 bmi 값으로 대체한 후 
# 대체된 bmi 컬럼의 평균을 소숫점 이하 3자리 까지 구하여라
tmp = data.groupby(data['age']//10*10)['bmi'].mean()
dic = { x:y for x,y in tmp.items()}

data.loc[data['bmi'].isnull()]['bmi']= (data[data['bmi'].isnull()]['age']//10*10).map(lambda x: dic[x])
result = data['bmi'].mean()
print(result)

29.2627029367386


In [230]:
# avg_glucose_level 컬럼의 값이 200이상인 데이터를 모두 199로 변경하고 
# stroke값이 1인 데이터의 avg_glucose_level값의 평균을 소수점이하 3자리 까지 구하여라

data.loc[data.avg_glucose_level >=200,'avg_glucose_level'] =199
ans = data[data['stroke']==1]['avg_glucose_level'].mean()
print(round(ans,3))

125.188


## 작업형 1 다른 데이터

In [285]:
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/pok/Pokemon.csv')
data.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [286]:
# Attack컬럼의 값을 기준으로 내림차순정렬 했을때 상위 400위까지 
# 포켓몬들과 401~800위까지의 포켓몬들에서 전설포켓몬(Legendary컬럼)의 숫자 차이는?
a = data[['Attack','Legendary']].sort_values('Attack',ascending=False).iloc[:400].groupby('Legendary').size().values[1]
b = data[['Attack','Legendary']].sort_values('Attack',ascending=False).iloc[401:].groupby('Legendary').size().values[1]
print(a-b)

57


In [287]:
# Type 1 컬럼의 종류에 따른 Total 컬럼의 평균값을 내림차순 정렬했을때 상위 3번째 Type 1은 무엇인가?

ans = data.groupby('Type 1')['Total'].mean().sort_values(ascending=False).index[2]
print(ans)

Flying


In [283]:
# 결측치가 존재하는 행을 모두 지운 후 처음부터 순서대로 60% 데이터를 추출하여 Defense컬럼의 1분위수를 구하여라

data.isnull().sum()
data.dropna(inplace=True)
tmp = data.head(60)
q1 = tmp['Defense'].quantile(.25)
print(q1)

50.0


In [288]:
# Type 1 컬럼의 속성이 Fire인 포켓몬들의 Attack의 평균이상인 Water속성의 포켓몬 수를 구하여라
rst = data[data['Attack']>=data[data['Type 1']=='Fire']['Attack'].mean()]
ans = rst[rst['Type 1']=='Water'].shape[0]
print(ans)

37


In [304]:
# 각 세대 중(Generation 컬럼)의 Speed와 Defense 컬럼의 차이(절댓값)이 가장 큰 세대는?

ans = abs(data[['Speed','Defense','Generation']].groupby('Generation').mean().T.diff().T.sort_values('Defense',ascending=False)).index[0]
print(ans)

2


## 작업형 2

In [335]:
import pandas as pd
train= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/stroke_/train.csv')
test= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/stroke_/test.csv')

display(train.head())
display(test.head())

y = train.pop('stroke')
train.drop('id',axis=1,inplace=True)
x_test_id = test.pop('id')

# print('-'*80)
# print(train.describe())
# print('-'*80)
# print(train.info())
# print('-'*80)
# print(train.isnull().sum())

# print('-'*80)
# print(test.describe())
# print('-'*80)
# print(test.info())
# print('-'*80)
# print(test.isnull().sum())

train['bmi'].fillna(train['bmi'].mean(),inplace=True)
test['bmi'].fillna(test['bmi'].mean(),inplace=True)

# print('-'*80)
# print(train.describe())
# print('-'*80)
# print(train.info())
# print('-'*80)
# print(train.isnull().sum())

# print('-'*80)
# print(test.describe())
# print('-'*80)
# print(test.info())
# print('-'*80)
# print(test.isnull().sum())

train['age']=train['age'].str.replace('*','').astype('int')
# test['age']=test['age'].str.replace('*','').astype('int')
test['age']=test['age'].astype('int')
print(train.info())
str_col = train.select_dtypes('object').columns
num_col = train.select_dtypes(exclude='object').columns

one_col = []
le_col = []
for col in str_col:
    print(f'{col} : \n {train[col].value_counts()}')
    print('-'*80)
    if train[col].nunique()<3:
        one_col.append(col)
    else:
        le_col.append(col)

from sklearn.preprocessing import StandardScaler,LabelEncoder

le = LabelEncoder()

for col in le_col:
    train[col] = le.fit_transform(train[col])
    test[col] = le.fit_transform(test[col])
train = pd.get_dummies(train,columns=one_col)
test = pd.get_dummies(test,columns=one_col)

scaler = StandardScaler()

for col in num_col:
    train[col] = scaler.fit_transform(train[[col]])
    test[col] = scaler.fit_transform(test[[col]])

from sklearn.model_selection import train_test_split

x_tr,x_te,y_tr,y_te = train_test_split(train,y,test_size=0.2,random_state=2022)

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

model_rf = RandomForestClassifier(random_state=2022)
model_rf.fit(x_tr,y_tr)
pred_rf = model_rf.predict(x_te)
score_rf = model_rf.score(x_te,y_te)

model_xgb = XGBClassifier(random_state=2022)
model_xgb.fit(x_tr,y_tr)
pred_xgb = model_xgb.predict(x_te)
score_xgb = model_xgb.score(x_te,y_te)

model_dt = DecisionTreeClassifier(random_state=2022)
model_dt.fit(x_tr,y_tr)
pred_dt = model_dt.predict(x_te)
score_dt = model_dt.score(x_te,y_te)

model_svc = SVC(random_state=2022)
model_svc.fit(x_tr,y_tr)
pred_svc = model_svc.predict(x_te)
score_svc = model_svc.score(x_te,y_te)

print(f'rf score : {score_rf}')
print(f'xgb score : {score_xgb}')
print(f'dt score : {score_dt}')
print(f'svc score : {score_svc}')

from sklearn.metrics import classification_report,roc_auc_score

report1 = classification_report(y_te,pred_rf)
report2 = classification_report(y_te,pred_xgb)
report3 = classification_report(y_te,pred_dt)
report4 = classification_report(y_te,pred_svc)

print(report1)
print(report2)
print(report3)
print(report4)

final_pred = model_rf.predict(test)
output=pd.DataFrame({'id':x_test_id,'stroke':final_pred})
output.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1192,Female,31,0,0,No,Govt_job,Rural,70.66,27.2,never smoked,0
1,77,Female,13,0,0,No,children,Rural,85.81,18.6,Unknown,0
2,59200,Male,18,0,0,No,Private,Urban,60.56,33.0,never smoked,0
3,24905,Female,65,0,0,Yes,Private,Urban,205.77,46.0,formerly smoked,1
4,24257,Male,4,0,0,No,children,Rural,90.42,16.2,Unknown,0


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,47472,Female,58,0,0,Yes,Private,Urban,107.26,38.6,formerly smoked
1,36841,Male,78,1,0,Yes,Self-employed,Rural,56.11,25.5,formerly smoked
2,3135,Female,73,0,0,No,Self-employed,Rural,69.35,NaN,never smoked
3,65218,Male,2,0,0,No,children,Rural,109.10,20.0,Unknown
4,1847,Female,20,0,0,No,Govt_job,Rural,79.53,NaN,never smoked


C:\Users\SuperUser\AppData\Local\Temp\ipykernel_11596\3159175129.py:43: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  train['age']=train['age'].str.replace('*','').astype('int')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1137 entries, 0 to 1136
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             1137 non-null   object 
 1   age                1137 non-null   int32  
 2   hypertension       1137 non-null   int64  
 3   heart_disease      1137 non-null   int64  
 4   ever_married       1137 non-null   object 
 5   work_type          1137 non-null   object 
 6   Residence_type     1137 non-null   object 
 7   avg_glucose_level  1137 non-null   float64
 8   bmi                1137 non-null   float64
 9   smoking_status     1137 non-null   object 
dtypes: float64(2), int32(1), int64(2), object(5)
memory usage: 84.5+ KB
None
gender : 
 Female    642
Male      494
Other       1
Name: gender, dtype: int64
--------------------------------------------------------------------------------
ever_married : 
 Yes    769
No     368
Name: ever_married, dtype: int64
------

C:\Users\SuperUser\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\SuperUser\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\SuperUser\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,id,stroke
0,47472,0
1,36841,0
2,3135,0
3,65218,0
4,1847,0


# 모의고사 3회

## 작업형 1(2010~2019) 스포티파이 TOP100 노래

In [336]:
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/spotify/spotify.csv')
data.head()

,title,artist,top genre,year released,added,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop,top year,artist type
0,STARSTRUKK (feat. Katy Perry),3OH!3,dance pop,2009.0,2022‑02‑17,140.0,81.0,61.0,-6.0,23.0,23.0,203.0,0.0,6.0,70.0,2010.0,Duo
1,My First Kiss (feat. Ke$ha),3OH!3,dance pop,2010.0,2022‑02‑17,138.0,89.0,68.0,-4.0,36.0,83.0,192.0,1.0,8.0,68.0,2010.0,Duo
2,I Need A Dollar,Aloe Blacc,pop soul,2010.0,2022‑02‑17,95.0,48.0,84.0,-7.0,9.0,96.0,243.0,20.0,3.0,72.0,2010.0,Solo
3,Airplanes (feat. Hayley Williams of Paramore),B.o.B,atl hip hop,2010.0,2022‑02‑17,93.0,87.0,66.0,-4.0,4.0,38.0,180.0,11.0,12.0,80.0,2010.0,Solo
4,Nothin' on You (feat. Bruno Mars),B.o.B,atl hip hop,2010.0,2022‑02‑17,104.0,85.0,69.0,-6.0,9.0,74.0,268.0,39.0,5.0,79.0,2010.0,Solo


In [344]:
# 데이터는 현재 년도별 100곡이 인기순으로 정렬되어 있다. 
# 각 년도별 1~100위의 랭킹을 나타내는 rank컬럼을 만들고 매년도 1위의 bpm컬럼의 평균값을 구하여라

data.dropna(inplace=True)
data.loc[:,'rank']=list(range(1,101))*10
ans = data[data['rank']==1]['bpm'].mean()
print(ans)

125.6


In [349]:
# 2015년도에 가장많은 top100곡을 올린 artist는 누구인가?
ans = data[data['top year']==2015]['artist'].value_counts().index[0]
print(ans)

Taylor Swift


In [361]:
# 년도별 rank값이 1~10위 까지의 곡들 중 두번째로 많은 top genre는 무엇인가?
ans = data[data['rank'].isin(range(1,11))]['top genre'].value_counts().index[2]
print(ans)

british soul


In [408]:
# 피처링의 경우 title에 표시된다. 피처링을 가장 많이 해준 가수는 누구인가?


ans = data['title'].str.split('feat.').str[1].dropna().str[:-1].value_counts().index[0]
print(ans)

 Bruno Mars


In [414]:
# top year 년도를 기준으로 발매일(year released)과 top100에 진입한 일자 (top year)가 다른 곡의 숫자를 count 했을때 가장 많은 년도는?

ans = int(data[data['top year']!=data['year released']]['top year'].value_counts().index[0])
print(ans)

2016


In [430]:
# artist 컬럼의 값에 대소문자 상관없이 q 단어가 들어가는 아티스트는 몇명인가?

ans = data[data['artist'].str.lower().str.contains('q')]['artist'].nunique()
print(ans)

6


In [440]:
# 년도 상관없이 전체데이터에서 1~50위와 51~100위간의 dur 컬럼의 평균값의 차이는?
ans = data[data['rank'].isin(range(1,51))]['dur'].mean()-data[data['rank'].isin(range(51,101))]['dur'].mean()
print(ans)

0.896000000000015


In [456]:
# title을 띄어쓰기 단어로 구분 했을때 가장 많이 나온 단어는 무엇인가? (대소문자 구분 x)
ans = data['title'].str.split('\(feat').str[0].str.split().explode().str.lower().value_counts().index[0]
print(ans)

the


In [460]:
# 년도별 nrgy값의 평균값을 구할때 최대 평균값과 최소 평균값의 차이를 구하여라
ans = data.groupby('top year')['nrgy'].mean().sort_values(ascending=False).max()-\
        data.groupby('top year')['nrgy'].mean().sort_values(ascending=False).min()
print(ans)

13.860000000000007


In [478]:
# artist중 artist type 타입을 여러개 가지고 있는 artist는 누구인가
ans = data[['artist','artist type']].value_counts().reset_index()['artist'].value_counts().index[0]
print(ans)

Rudimental


## 작업형 2

In [492]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/muscle/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/muscle/y_train.csv")
test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/muscle/x_test.csv")


display(x_train.head())
display(y_train.head())

x_test_id = test.pop('ID')
x_train.drop('ID',axis=1,inplace=True)
y=y_train['pose']

from sklearn.preprocessing import StandardScaler

scale =StandardScaler()
x_train = scale.fit_transform(x_train)
test = scale.fit_transform(test)

from sklearn.model_selection import train_test_split

x_tr,x_te,y_tr,y_te = train_test_split(x_train,y,stratify=y,random_state=2022)
x_tr.shape,x_te.shape,y_tr.shape,y_te.shape

from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(random_state=2022)
model_rf.fit(x_tr,y_tr)
pred_rf = model_rf.predict(x_te)
prob_rf = model_rf.predict_proba(x_te)
score_rf = model_rf.score(x_te,y_te)

from xgboost import XGBClassifier
model_xg = XGBClassifier(random_state=2022)
model_xg.fit(x_tr,y_tr)
pred_xg = model_xg.predict(x_te)
prob_xg = model_xg.predict_proba(x_te)
score_xg =model_xg.score(x_te,y_te)

from sklearn.svm import SVC

model_sv = SVC(probability=True,random_state=2022)
model_sv.fit(x_tr,y_tr)
pred_sv = model_sv.predict(x_te)
prob_sv = model_sv.predict_proba(x_te)
score_sv = model_sv.score(x_te,y_te)

print(score_rf)
print(score_xg)
print(score_sv)

from sklearn.metrics import classification_report, roc_auc_score

report1 = classification_report(y_te,pred_rf)
report2 = classification_report(y_te,pred_xg)
report3 = classification_report(y_te,pred_sv)

roc1 = roc_auc_score(y_te,prob_rf[:,1])
roc2 = roc_auc_score(y_te,prob_xg[:,1])
roc3 = roc_auc_score(y_te,prob_sv[:,1])

print(report1)
print(report2)
print(report3)
print('--'*80)
print(roc1)
print(roc2)
print(roc3)

,ID,motion_0,motion_1,motion_2,motion_3,motion_4,motion_5,motion_6,motion_7,motion_8,...,motion_54,motion_55,motion_56,motion_57,motion_58,motion_59,motion_60,motion_61,motion_62,motion_63
0,0,1.0,-2.0,-1.0,4.0,-5.0,-4.0,1.0,0.0,-15.0,...,0.0,-1.0,-13.0,-3.0,1.0,-1.0,-32.0,-22.0,-2.0,-3.0
1,2,20.0,0.0,0.0,1.0,5.0,6.0,-52.0,18.0,15.0,...,-70.0,-55.0,-38.0,-14.0,-12.0,-8.0,-34.0,-63.0,-87.0,-77.0
2,4,1.0,-1.0,1.0,4.0,-5.0,-8.0,1.0,-3.0,-14.0,...,1.0,12.0,-25.0,0.0,0.0,3.0,2.0,-27.0,1.0,0.0
3,5,13.0,2.0,1.0,-3.0,1.0,3.0,28.0,3.0,12.0,...,0.0,-21.0,-17.0,-2.0,0.0,-4.0,-17.0,-21.0,-21.0,25.0
4,6,-2.0,-7.0,-4.0,-8.0,16.0,44.0,1.0,3.0,-16.0,...,-1.0,2.0,-1.0,1.0,4.0,4.0,-17.0,-38.0,-3.0,3.0


,ID,pose
0,0,1
1,2,0
2,4,1
3,5,0
4,6,1


0.9914015477214101
0.9974204643164231
0.9819432502149613
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       582
           1       1.00      0.98      0.99       581

    accuracy                           0.99      1163
   macro avg       0.99      0.99      0.99      1163
weighted avg       0.99      0.99      0.99      1163

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       582
           1       1.00      0.99      1.00       581

    accuracy                           1.00      1163
   macro avg       1.00      1.00      1.00      1163
weighted avg       1.00      1.00      1.00      1163

              precision    recall  f1-score   support

           0       0.99      0.97      0.98       582
           1       0.97      0.99      0.98       581

    accuracy                           0.98      1163
   macro avg       0.98      0.98      0.98      1163
weighted avg     

In [487]:
print(score_rf)

0.9914015477214101


In [493]:
prob_rf[:,1]

array([0.02, 0.01, 1.  , ..., 0.01, 0.99, 0.99])